In [81]:
import cv2,numpy as np

In [130]:
src = cv2.imread('hackathon_2022-main/good.png')

In [131]:
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
gray = cv2.resize(gray, (500,500), interpolation = cv2.INTER_AREA)

In [132]:
# Performing Canny edge detection
gray_canny = cv2.Canny(gray,40,120)

In [133]:
# rows = gray.shape[0]
# Concentric circles cannot be detected at one so first detecting the outer circle
circles = cv2.HoughCircles(gray_canny, cv2.HOUGH_GRADIENT, 1, 500,
                               param1=140, param2=10,
                               minRadius=1, maxRadius=400)
outer_circle = circles

In [134]:
# Masking the outer circle
circle = outer_circle
if circles is not None:
    circles = np.uint16(np.around(circles))
    for i in circles[0, :]:
        center = (i[0], i[1])
        radius = i[2]
        cv2.circle(gray_canny, center, radius, (0, 0, 0), 7)

In [135]:
# Detecting the inner circle
#rows = gray.shape[0]
circles = cv2.HoughCircles(gray_canny, cv2.HOUGH_GRADIENT, 1, 500,
                               param1=140, param2=10,
                               minRadius=1, maxRadius=400)
inner_circle = circles

In [136]:
# Masking inner circle
circle = inner_circle
if circles is not None:
    circles = np.uint16(np.around(circles))
    for i in circles[0, :]:
        center = (i[0], i[1])
        radius = i[2]
        cv2.circle(gray_canny, center, radius, (0, 0, 0), 7)

In [140]:
# Combining the detected inner circle and outer circle into single numpy array
combined_circle = np.append(inner_circle,outer_circle,axis = 1)

In [141]:
# Finding the points which are not on circle from
res = np.where(gray_canny > 250)

In [142]:
# Calculating the distance between the center and the points detects
# Checking the defect by comparing the distance calculated with radiuss
import math
def calculateDistance(x1,y1,x2,y2):
    dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return dist

output = {'Flash':0,'Cut':0}

for (x1,y1) in zip(res[0],res[1]):
    x2_i,y2_i = combined_circle[0][0][0], combined_circle[0][0][1]
    x2_o,y2_o = combined_circle[0][1][0], combined_circle[0][1][1]
    
    inner_compare = calculateDistance(x1, y1, x2_i, y2_i)
    outer_compare = calculateDistance(x1, y1, x2_o, y2_o)
    
    if ((inner_compare < combined_circle[0][0][2]) or  (outer_compare > combined_circle[0][1][2])):
        output['Flash'] = output['Flash'] + 1
    if ((inner_compare > combined_circle[0][0][2]) & (outer_compare < combined_circle[0][1][2])):
        output['Cut'] = output['Cut'] + 1

if output ['Cut'] > 5:
    print('Defect - Cut')
if output['Flash'] > 5:
    print('Defect - Flash')
if ((output['Cut']+output['Flash']) < 2):
    print('No Defects - Good')

No Defects - Good
